# **Panzura Use Case 1 - Identify too many reads**

<font color=red>please make this confidential</font>

- <a href='#1'>1. EDA - Understanding the data</a>
    - <a href='#1.1'>1.1 Second-wise frequency</a>
    - <a href='#1.2'>1.2 Minute-wise frequency</a>
    - <a href='#1.3'>1.3 Hour-wise frequency</a>
- <a href='#2'>2. Train a model</a>
- <a href='#3'>3. Anomaly Detection</a>
- <a href='#4'>4. Who has most reads during abnormal period ?</a>
- <a href='#5'>5. Which file was accessed mostly ?


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go


In [ ]:
#file_path = '/kaggle/input/panzura-cc-log/file_audit_0901_0916.csv'
file_path = '/kaggle/input/panzura-cc-log/parsed_result_201910.csv'

# <a id='1'> 1. EDA - Understanding the data

In [ ]:
#df_origin = pd.read_csv(file_path, keep_default_na = False)
#df_origin[df_origin.file.isnull()]

In [ ]:
#df_origin[df_origin.timestamp == 1568241707]

In [ ]:
import datetime

def date_parser (time_in_secs):
    return datetime.datetime.fromtimestamp(float(time_in_secs))

df_origin = pd.read_csv(file_path, parse_dates=[4], date_parser=date_parser, 
                 index_col='timestamp', keep_default_na = False)

df_origin.tail()

In [ ]:
#df_origin.sort_index().head()

In [ ]:
#df_origin.sort_index().tail()

In [ ]:
# check if there is null value
#df_origin.isnull().sum()

In [ ]:
#temp = df_origin[df_origin.file.isnull()]
#temp

In [ ]:
df_origin['open_counts'] = 1
#df_origin.dtypes

In [ ]:
len(df_origin.user.unique())

## <a id='1.1'>1.1 Second-wise frequency

In [ ]:
df = df_origin.resample('s').sum()
#df.tail()

In [ ]:
_ = df['open_counts'].plot(figsize=(20, 5))
plt.title('Open Operation Counts(Secondly frequency)')
plt.show()

## <a id='1.2'>1.2 Minute-wise frequency

In [ ]:
temp = df['open_counts'].resample('T').sum()
temp.plot(figsize=(20, 5))
plt.title('Open Operation Counts(Minutely frequency)')
plt.show()

## <a id='1.3'>1.3 Hour-wise frequency

In [ ]:
temp = df['open_counts'].resample('H').sum()
temp.plot(figsize=(20, 5))
plt.title('Open Operation Counts(Hourly frequency)')
plt.show()

temp = df['open_counts'].resample('D').sum()
temp.plot(figsize=(20, 5))
plt.title('Open Operation Counts(Daily frequency)')
plt.show()

In [ ]:
#temp = df['open_counts'].resample('3H').sum()

df = temp.to_frame()
#df.head()

In [ ]:
#df.isnull().sum()

In [ ]:
#!pip install chart_studio
#import chart_studio.plotly as py


#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#init_notebook_mode(connected=True)


#fig = go.Figure(go.Scatter(x=df.index, y=df.open_counts, mode='markers')
#fig = go.Figure([go.Scatter(x=df.index, y=df['open_counts'])])
#fig.update_layout(title='Open Operation Counts')
#fig.show()

#fig.update_xaxes(title_text='GDP per Capita', type='log')
#fig.update_yaxes(title_text='Life Expectancy')

#py.iplot(fig, filename='pandas-multiple-scatter')

## <a id='2'>2. Train a model

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs(y_true - y_pred) / y_true) * 100

In [ ]:
clean_df = df.reset_index().rename(columns={"timestamp": "ds", "open_counts": "y"})
clean_df

In [ ]:
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
import time

m = Prophet(seasonality_prior_scale=10.0,
            interval_width=0.95,
            changepoint_prior_scale=10,
            weekly_seasonality=False,
            daily_seasonality=False
            ).add_seasonality(name='daily', period=1, fourier_order=10
            ).add_seasonality(name='weekly', period=7, fourier_order=5)

start = time.time()

m.fit(clean_df)

period = time.time() - start
print('traing time for model:  {}'.format(period))


In [ ]:
future = m.make_future_dataframe(periods=1)
future.tail()

In [ ]:
forecast = m.predict(future)

In [ ]:
m.plot_components(forecast);


In [ ]:
pd.plotting.register_matplotlib_converters()

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
from fbprophet.plot import plot_plotly
import plotly.offline as py
py.init_notebook_mode()

#fig = plot_plotly(m, forecast)  # This returns a plotly Figure
#py.iplot(fig)

## <a id='3'>3. Anomaly Detection

In [ ]:
def detect_anomalies(forecast):
    forecasted = forecast[['ds','trend', 'yhat', 'yhat_lower', 'yhat_upper', 'fact']].copy()
    #forecast['fact'] = df['y']

    forecasted['anomaly'] = 0
    forecasted.loc[forecasted['fact'] > forecasted['yhat_upper'], 'anomaly'] = 1
    forecasted.loc[forecasted['fact'] < forecasted['yhat_lower'], 'anomaly'] = -1

    #anomaly importances
    forecasted['importance'] = 0
    forecasted.loc[forecasted['anomaly'] ==1, 'importance'] = \
        (forecasted['fact'] - forecasted['yhat_upper'])/forecast['fact']
    forecasted.loc[forecasted['anomaly'] ==-1, 'importance'] = \
        (forecasted['yhat_lower'] - forecasted['fact'])/forecast['fact']
    
    return forecasted

forecast['fact'] = clean_df['y'].reset_index(drop = True)

pred = detect_anomalies(forecast)
#pred

In [ ]:
#mape_baseline = mean_absolute_percentage_error(forecast.fact, forecast.yhat)
#mape_baseline

In [ ]:
#forecast.yhat

In [ ]:
# post-processing

# similate another outlier


# set floor
floor = 0
pred['yhat'] = pred['yhat'].apply(lambda x: floor if x<floor else x)
pred['yhat_lower'] = pred['yhat_lower'].apply(lambda x: floor if x<floor else x)

#pred[pred['anomaly'] == 1]

In [ ]:
def plot_anomalies(forecast):
    # Create figure
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(x=forecast.loc[forecast['anomaly'] == 1, 'ds'], y=forecast.loc[forecast['anomaly'] == 1, 'fact'], name='Anomaly',
                   mode='markers',
                   marker_color='rgba(255, 0, 0, .7)',
                   marker_size=forecast.loc[forecast['anomaly'] == 1, 'importance']*50))
    fig.add_trace(
        go.Scatter(x=forecast.loc[forecast['anomaly'] != 1, 'ds'], y=forecast.loc[forecast['anomaly'] != 1, 'fact'], name='Normal',
                   mode='markers', marker_color='rgba(0, 0, 255, .8)'))

    fig.add_trace(
        go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='yhat'))
    fig.add_trace(
        go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], fill='tonexty', mode='none', name='upper'))
    fig.add_trace(
        go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill='tonexty', mode='none', name='lower'))
    #fig.add_trace(
    #    go.Scatter(x=forecast['ds'], y=forecast['trend'], name='Trend'))

    # Set title
    fig.update_layout(
        title_text="Time series with range slider and selectors"
    )

    # Add range slider
    fig.update_layout(
        xaxis=go.layout.XAxis(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label="1m",
                         step="month",
                         stepmode="backward"),
                    dict(count=6,
                         label="6m",
                         step="month",
                         stepmode="backward"),
                    dict(count=1,
                         label="YTD",
                         step="year",
                         stepmode="todate"),
                    dict(count=1,
                         label="1y",
                         step="year",
                         stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(
                visible=True
            ),
            type="date"
        )
    )

    fig.show()
    
    
plot_anomalies(pred)

# <a id='4'> 4. who has most reads during abnormal period ?

In [ ]:
suspicious_range = ['2019-09-27', '2019-09-28']

df_reduced = df_origin.loc[df_origin.index < suspicious_range[1]].copy()
df_reduced = df_reduced.loc[df_reduced.index > suspicious_range[0]].copy()

#df_reduced.shape

In [ ]:
import seaborn as sns

tmp = df_reduced.user.value_counts()[:10]
#tmp


In [ ]:


plt.figure(figsize=(8,5))
sns.barplot(tmp.index, tmp.values, alpha=0.8)
plt.title('Who access most?')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('user', fontsize=12)
plt.xticks(rotation=80)
plt.show()

In [ ]:

#tmp = df_reduced.reset_index()
#tmp

# <a id='5'> 5. Which file was accessed mostly ?

In [ ]:
tmp = df_reduced.file.value_counts()[:10]
#tmp

In [ ]:

plt.figure(figsize=(8,5))
sns.barplot(tmp.index, tmp.values, alpha=0.8)
plt.title('Which file was accessed mostly by all users?')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('file', fontsize=12)
plt.xticks(rotation=80)
plt.show()

In [ ]:
tmp = df_reduced.user.value_counts()[:10]


In [ ]:
haha = df_reduced[df_reduced.user == tmp.index[0]].file.value_counts()[:10]
#haha

In [ ]:
plt.figure(figsize=(8,5))
sns.barplot(haha.index, haha.values, alpha=0.8)
plt.title('Which file was accessed mostly by the specific user?')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('file', fontsize=12)
plt.xticks(rotation=80)
plt.show()